In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/deepdaiv위급상황탐지/EfficientAT')

In [ ]:
demo_path1 = '/content/drive/MyDrive/deepdaiv위급상황탐지/EfficientAT/데모/소리 녹음'
demo_list1 = os.listdir(demo_path1)
demo_list1.remove('.ipynb_checkpoints')

In [ ]:
print(demo_list1)

In [ ]:
demo_path2 = '/content/drive/MyDrive/deepdaiv위급상황탐지/EfficientAT/데모/Ai Hub 위급상황'
demo_list2 = os.listdir(demo_path2)
demo_list2.remove('.ipynb_checkpoints')

In [ ]:
print(demo_list2)

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import csv

import matplotlib.pyplot as plt
from IPython.display import Audio
from scipy.io import wavfile
import librosa

def screamdetect(audio_path,threshold,screamdetecting):
  yamnet = hub.load('https://tfhub.dev/google/yamnet/1')

  # Find the name of the class with the top score when mean-aggregated across frames.
  def class_names_from_csv(class_map_csv_text):
    """Returns list of class names corresponding to score vector."""
    class_names = []
    with tf.io.gfile.GFile(class_map_csv_text) as csvfile:
      reader = csv.DictReader(csvfile)
      for row in reader:
        class_names.append(row['display_name'])

    return class_names

  def split_wav_data(wav_data, sample_rate, segment_length=1):
    """Split the wav data into segments of specified length."""
    num_samples_per_segment = sample_rate * segment_length
    num_segments = int(np.ceil(len(wav_data) / num_samples_per_segment))
    segments = []

    for i in range(num_segments):
        start = i * num_samples_per_segment
        end = start + num_samples_per_segment
        segment = wav_data[start:end]
        segments.append(segment)

    return segments


  class_map_path = yamnet.class_map_path().numpy()
  class_names = class_names_from_csv(class_map_path)
  waveform , _ = librosa.core.load(audio_path,sr=16000,mono=True)
  waveform = waveform.astype('float16')
  segments = split_wav_data(waveform, 16000)
  print('segments:',len(segments))

  threshold = threshold
  scream_idx = []
  for i,wave in enumerate(segments):
      scores, embeddings, spectrogram = yamnet(waveform)

    # 결과를 numpy 배열로 변환
      scores_np = scores.numpy()
      spectrogram_np = spectrogram.numpy()

      # 임계값을 초과하는 모든 클래스 식별
      detected_classes = [class_names[idx] for idx, score in enumerate(scores_np.mean(axis=0)) if score > threshold]
      if 'Screaming' in detected_classes:
        scream_idx.append((i))
        print(f'Segment {i}: Scream Detected')

  if screamdetecting == True:
    if len(scream_idx) == 0:
      return False #비명이 감지되지않으면 일반상황판단

    elif len(scream_idx) == len(segments):
      scream_point = 0 #비명이 모든 오디오에서 감지되면 모든 오디오 위급상황 판단

    else:
      for i in range(len(scream_idx)-1):
        if scream_idx[i+1] - scream_idx[i] != 1:
          scream_point = scream_idx[i] #첫번째 비명이후부터 위급상황판단
          break
  else:
    scream_point = 0

  sampling_rate = 32000
  waveform2 , _ = librosa.core.load(audio_path,sr=sampling_rate
                                 ,mono=True)

  segments2 = split_wav_data(waveform2,sampling_rate)

  #비명이후 위급상황감지 슬라이딩 윈도우

  clf_leng = len(segments2)-1 - scream_point
  print('clf_leng',clf_leng)
  #print(clf_leng)
  wave_list = []
  if clf_leng == 0:
    if len(segments2) > 8:
      additional = len(segments2) - 8
      scream_point = 0
      for i in list(range(additional+1)):
        wave = np.concatenate(segments2[scream_point+i:scream_point+8+i])
        wave_list.append(wave)
        #print(len(wave_list))
    else:
        wave = np.concatenate(segments2)
        wave_list.append(wave)

  elif clf_leng > 8:
    additional = clf_leng - 8
    for i in list(range(additional+1)):
      wave = np.concatenate(segments2[scream_point+i:scream_point+8+i])
      wave_list.append(wave)
  else:
    wave = np.concatenate(segments2[scream_point+1:])
    wave_list.append(wave)

  return wave_list





In [ ]:
import argparse
import torch
import librosa
import numpy as np
import os
from torch import autocast
from contextlib import nullcontext

from models.mn.model_nonprint import get_model as get_mobilenet
from models.dymn.model import get_model as get_dymn
from models.ensemble import get_ensemble_model
from models.preprocess import AugmentMelSTFT
from helpers.utils import NAME_TO_WIDTH, labels


def audio_tagging(args,wave_list):
    if wave_list == False:
      return print('현재상황:정상')

    print('num of wave',len(wave_list))


    """
    Running Inference on an audio clip.
    """
    model_name = args.model_name
    device = torch.device('cuda') if args.cuda and torch.cuda.is_available() else torch.device('cpu')
    audio_path = args.audio_path
    sample_rate = args.sample_rate
    window_size = args.window_size
    hop_size = args.hop_size
    n_mels = args.n_mels
    model_path = args.model_path
    labels = ['강제추행(성범죄)', '강도범죄', '절도범죄', '폭력범죄',
              '화재', '갇힘', '응급의료',
              '전기사고', '가스사고', '낙상', '붕괴사고', '도움요청']

    # load pre-trained model
    '''if len(args.ensemble) > 0:
        model = get_ensemble_model(args.ensemble)
    else:'''
    if model_name.startswith("dymn"):
        model = get_dymn(width_mult=NAME_TO_WIDTH(model_name), pretrained_name=model_name,
                                  strides=args.strides,num_classes=12)
    else:
        #print('check1')
        model = get_mobilenet(width_mult=NAME_TO_WIDTH(model_name), pretrained_name=model_name,
                                  strides=args.strides, head_type=args.head_type,se_dims=args.se_dims,
                                  num_classes=12)
    model.to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    # model to preprocess waveform into mel spectrograms
    mel = AugmentMelSTFT(n_mels=n_mels, sr=sample_rate, win_length=window_size, hopsize=hop_size)
    mel.to(device)
    mel.eval()

    for i in range(len(wave_list)):
      waveform = wave_list[i]
      waveform = waveform.astype(np.float32)
      #(waveform, _) = librosa.core.load(audio_path, sr=sample_rate, mono=True)
      waveform = torch.from_numpy(waveform[None, :]).to(device)

      # our models are trained in half precision mode (torch.float16)
      # run on cuda with torch.float16 to get the best performance
      # running on cpu with torch.float32 gives similar performance, using torch.bfloat16 is worse
      with torch.no_grad(), autocast(device_type=device.type) if args.cuda else nullcontext():
          spec = mel(waveform)
          preds, features = model(spec.unsqueeze(0))
      if i == 0:
        predsum = preds

        pred_idx = torch.argmax(preds,dim = 1)
        situation = labels[pred_idx]
        print("************* Detecting *****************")
        print(f'Window {i+1} 상황:{situation}')
      else:
        predsum += preds

        pred_idx = torch.argmax(preds,dim = 1)
        situation = labels[pred_idx]
        print("************* Detecting *****************")
        print(f'Window {i+1} 상황:{situation}')

    pred_idx = torch.argmax(predsum,dim = 1)
    situation = labels[pred_idx]
    print("************* Result *****************")
    print(f'현재 상황:{situation}')

    #preds = torch.sigmoid(preds.float()).squeeze().cpu().numpy()
    #sorted_indexes = np.argsort(preds)[::-1]

    '''# Print audio tagging top probabilities
    print("************* Acoustic Event Detected: *****************")
    for k in range(10):
        print('{}: {:.3f}'.format(labels[sorted_indexes[k]],
            preds[sorted_indexes[k]]))
    print("********************************************************")'''


In [ ]:
def emergencydetect(args):
  wave_list = screamdetect(args.audio_path,threshold=args.threshold,screamdetecting=args.screamdetecting)
  audio_tagging(args,wave_list)

#데모

In [ ]:
!pip install -q IPython

In [ ]:
#영화 오디오
import random
idx = random.randint(0,len(demo_list1))
#random_demo = demo_list1[idx]
#['붕괴.wav', '절도 - 도깨비.wav', '폭행 - 수리남.wav', '화재 - 뉴스.wav', '강도 - 아는 와이프.wav']
random_demo = demo_list1[3]
audio_path = demo_path1 + '/' + random_demo
print('Selected demo:',idx)

In [ ]:
#Ai-Hub 위급상황데이터
import random
idx = random.randint(0,len(demo_list2))
random_demo = demo_list2[idx]
audio_path = demo_path2 + '/' + random_demo
print('Selected demo:',idx)

In [ ]:
from IPython.display import Audio

Audio(audio_path)

In [ ]:
import easydict

args = easydict.EasyDict({'model_name':'mn10_as',
                          'strides': [2,2,2,2],
                          'head_type':'mlp',
                          'se_dims':"c",
                          'sample_rate':32000,
                          'window_size':800,
                          'hop_size':320,
                          'n_mels':128,
                          'cuda':True,
                          'ensemble':[],
                          'screamdetecting': False,
                          'threshold': 0.0001,
                          'audio_path': audio_path,
                          'model_path':'/content/drive/MyDrive/deepdaiv위급상황탐지/EfficientAT/inference_test/mn10_custom_12class_epoch_14_mAP_100.pt'})

In [ ]:
emergencydetect(args)
#print(random_demo)